In [1]:
import numpy
import numpy as np
import pandas
import pandas as pd
from numba import njit
import sys,glob
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
import plotly.express as px
sys.path.append(os.getcwd()+'/../cds-backend/code/')
sys.path.append(os.getcwd()+'/../harvest/code/')
from harvest_convert_to_netCDF_newfixes import write_dict_h5
import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
#import schedule
import copy
from shutil import copyfile
import multiprocessing

In [2]:
files = glob.glob('/raid60/scratch/leo/scratch/RI/Pangaea/nc/0*.nc')
# files = glob.glob('/raid60/scratch/uli/0*.nc')
files[0]

'/raid60/scratch/leo/scratch/RI/Pangaea/nc/0-20200-0-01501.nc'

In [3]:
def do_resort(fn):
    targetfile = '/raid60/scratch/uli/resorted_files/'+fn.split('/')[-1]  
    
    with h5py.File(fn, 'r') as file:
        with h5py.File(targetfile, 'w') as newfile:
            groups = []
            for i in file.keys():
                if type(file[i]) == h5py._hl.group.Group:
                    newfile.create_group(i)
                    groups.append(i)
                elif i == 'recordindex' or i == 'recordtimestamp':
                    pass
                else:
                    newfile.create_dataset(i, data=file[i][:])
            for i in groups:
                if(i == 'recordindices' or i == 'observations_table' or i == 'era5fb'):
                    pass
                else:
                    for j in file[i].keys():
                        newfile[i].create_dataset(j, data=file[i][j][:])
    
    data =  eua.CDMDataset(fn)
    allvars = data.observations_table.observed_variable[()]
    allvars.sort()
    allvars = numpy.unique(allvars)
    #
    ri = data.recordindex[()]
#     print('recordindex: ', len(ri))
    rt = data.recordtimestamp[()]
    keys = data.observations_table.keys()[:-1]
    fbkeys = data.era5fb.keys()[:-1]
    # dropping all keys, where dimensions won't work - just help variabels for dimensions
    pops = []
    for i in range(len(keys)):
        if 'string' in keys[i]:
            pops.append(keys[i])
    for i in pops: keys.remove(i)
    pops = []
    for i in range(len(fbkeys)):
        if 'string' in fbkeys[i]:
            pops.append(fbkeys[i])
    for i in pops: fbkeys.remove(i)

    recordindices = [[] for i in range(len(allvars))]
    recordtimestamps = [[] for i in range(len(allvars))]

    # output variables (from observations_table)
    ov = []
    for o in keys:
        ov.append([[] for i in range(len(allvars))])
    fb = []
    for o in fbkeys:
        fb.append([[] for i in range(len(allvars))])
    #
    # loading the observed_variables
    #
    obsv = data.observations_table.observed_variable[:]
    #
    # resorting the data
    #
#     print('resort:start')
    @njit
    def make_vrindex(vridx,ridx,idx):
        l=0
        for i in range(1,len(idx)): # to set the recordindices
            if ridx[i]>ridx[i-1]:
                vridx[ridx[i-1]:ridx[i]]=l # next record after l
                l=i
        vridx[ridx[i]:]=len(idx) # next record for the last element is the len of the data


    tt=time.time()

    ridxall=np.zeros(obsv.shape[0],dtype=np.int64) # reverse index - index of the record index
    j=-1
    for j in range(len(ri)-1):
        ridxall[ri[j]:ri[j+1]]=j
    j+=1
    ridxall[ri[j]:]=j # for the last elemenet
    ridx=[]
    vridx=[]
    absidx=[]
    abscount=0
    for j in range(len(allvars)):
        idx=np.where(obsv==allvars[j])[0] # index of all elements form certain variable j
#         print(j,len(idx),',',end='')
        vridx.append(np.zeros(ri.shape[0],dtype=np.int64)) # all zeros in lenght of record index
        ridx=ridxall[idx] # ridxall where variable is j
        make_vrindex(vridx[-1],ridx,idx)
        vridx[-1]+=abscount # abscount for stacking the recordindex

        absidx.append(copy.copy(idx)) # why copy? - to make sure it's not just the ref. - maybe ok without the cp
        abscount+=len(idx)

#     print('')
    #
    # finishing the sorting 
    #
    absidx=np.concatenate(absidx)
    #
    # recordtimestamps are only necessary once
    #
    recordtimestamps = recordtimestamps[0]
    #
    # targetfile has to be a copy of the original file
    #
    targetfile = '/raid60/scratch/uli/resorted_files/'+fn.split('/')[-1]# 0-20000-0-63894_CEUAS_merged_v0.nc'
    if os.path.isfile(targetfile):
        mode='r+'
    else:
        mode='w'
#     print()
#     print('writing '+targetfile)

    for i in range(len(keys)):
        ov_vars = data.observations_table[keys[i]][:]
        ov_vars = ov_vars[absidx]
        if keys[i] == 'index':
            pass
        elif keys[i] == 'observation_id' or keys[i] == 'report_id' or keys[i] == 'sensor_id' or keys[i] == 'source_id':
            alldict = {keys[i]:np.asarray(ov_vars, dtype='S1')}
            write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])
        else:
            alldict = pandas.DataFrame({keys[i]:ov_vars})
            write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])  

    for i in range(len(fbkeys)):
        fb_vars = data.era5fb[fbkeys[i]][:]
        fb_vars = fb_vars[absidx]
        if fbkeys[i] == 'index' or fbkeys[i] == 'string6' or fbkeys[i] == 'string7' or fbkeys[i] == 'string10':
            pass
        elif fbkeys[i] == 'expver' or fbkeys[i] == 'source@hdr' or fbkeys[i] == 'source_id' or fbkeys[i] == 'statid@hdr':
            alldict = {fbkeys[i]:np.asarray(fb_vars, dtype='S1')}
            write_dict_h5(targetfile, alldict, 'era5fb', {fbkeys[i]: { 'compression': 'gzip' } }, [fbkeys[i]])
        else:
            alldict = pandas.DataFrame({fbkeys[i]:fb_vars})
            write_dict_h5(targetfile, alldict, 'era5fb', {fbkeys[i]: { 'compression': 'gzip' } }, [fbkeys[i]]) 
    #
    # writing the recordindices and recordtimestamp.
    #       
    recordindices=vridx
    for i in range(len(recordindices)):
        testvar = pandas.DataFrame({str(allvars[i]):recordindices[i]})
        write_dict_h5(targetfile, testvar, 'recordindices', {str(allvars[i]): { 'compression': None } }, [str(allvars[i])]) 

    write_dict_h5(targetfile, {'recordtimestamp':rt}, 'recordindices', {'recordtimestamp': { 'compression': None } }, ['recordtimestamp'])

    print('elapsed:',time.time()-tt)



In [93]:
def do_resort(fn):
    targetfile = '/raid60/scratch/uli/resorted_files_intercomp/'+fn.split('/')[-1]  
    
    with h5py.File(fn, 'r') as file:
        with h5py.File(targetfile, 'w') as newfile:
            groups = []
            for i in file.keys():
                if type(file[i]) == h5py._hl.group.Group:
                    newfile.create_group(i)
                    groups.append(i)
                elif i == 'recordindex' or i == 'recordtimestamp':
                    pass
                else:
                    newfile.create_dataset(i, data=file[i][:])
            for i in groups:
                if(i == 'recordindices' or i == 'observations_table' or i == 'era5fb'):
                    pass
                else:
                    for j in file[i].keys():
                        newfile[i].create_dataset(j, data=file[i][j][:])
    

    data =  eua.CDMDataset(fn)
    allvars = data.observations_table.observed_variable[()]
    allvars.sort()
    allvars = numpy.unique(allvars)
    #
    ri = data.recordindex[()]
#     print('recordindex: ', len(ri))
    rt = data.recordtimestamp[()]
    keys = data.observations_table.keys()#[:-1]
#     fbkeys = data.era5fb.keys()[:-1]
    # dropping all keys, where dimensions won't work - just help variabels for dimensions
    pops = []
    for i in range(len(keys)):
        if 'string' in keys[i]:
            pops.append(keys[i])
    for i in pops: keys.remove(i)
    keys.remove('shape')
#     pops = []
#     for i in range(len(fbkeys)):
#         if 'string' in fbkeys[i]:
#             pops.append(fbkeys[i])
#     for i in pops: fbkeys.remove(i)

    recordindices = [[] for i in range(len(allvars))]
    recordtimestamps = [[] for i in range(len(allvars))]

    # output variables (from observations_table)
    ov = []
    for o in keys:
        ov.append([[] for i in range(len(allvars))])
#     fb = []
#     for o in fbkeys:
#         fb.append([[] for i in range(len(allvars))])
    #
    # loading the observed_variables
    #
    obsv = data.observations_table.observed_variable[:]
    #
    # resorting the data
    #
#     print('resort:start')
    @njit
    def make_vrindex(vridx,ridx,idx):
        l=0
        for i in range(1,len(idx)): # to set the recordindices
            if ridx[i]>ridx[i-1]:
                vridx[ridx[i-1]:ridx[i]]=l # next record after l
                l=i
        vridx[ridx[i]:]=len(idx) # next record for the last element is the len of the data


    tt=time.time()

    ridxall=np.zeros(obsv.shape[0],dtype=np.int64) # reverse index - index of the record index
    j=-1
    for j in range(len(ri)-1):
        ridxall[ri[j]:ri[j+1]]=j
    j+=1
    ridxall[ri[j]:]=j # for the last elemenet
    ridx=[]
    vridx=[]
    absidx=[]
    abscount=0
    for j in range(len(allvars)):
        idx=np.where(obsv==allvars[j])[0] # index of all elements form certain variable j
#         print(j,len(idx),',',end='')
        vridx.append(np.zeros(ri.shape[0],dtype=np.int64)) # all zeros in lenght of record index
        ridx=ridxall[idx] # ridxall where variable is j
        make_vrindex(vridx[-1],ridx,idx)
        vridx[-1]+=abscount # abscount for stacking the recordindex

        absidx.append(copy.copy(idx)) # why copy? - to make sure it's not just the ref. - maybe ok without the cp
        abscount+=len(idx)

#     print('')
    #
    # finishing the sorting 
    #
    absidx=np.concatenate(absidx)
    #
    # recordtimestamps are only necessary once
    #
    recordtimestamps = recordtimestamps[0]
    #
    # targetfile has to be a copy of the original file
    #
#     targetfile = '/raid60/scratch/uli/resorted_files/'+fn.split('/')[-1]# 0-20000-0-63894_CEUAS_merged_v0.nc'
    if os.path.isfile(targetfile):
        mode='r+'
    else:
        mode='w'
#     print()
#     print('writing '+targetfile)

    for i in range(len(keys)):
        ov_vars = data.observations_table[keys[i]][:]
        ov_vars = ov_vars[absidx]
        if keys[i] == 'index':
            pass
        elif keys[i] == 'observation_id' or keys[i] == 'report_id' or keys[i] == 'sensor_id' or keys[i] == 'reference_sensor_id' or keys[i] == 'adjustment_id':
            alldict = {keys[i]:np.asarray(ov_vars, dtype='S1')}
            write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])
        else:
            alldict = pandas.DataFrame({keys[i]:ov_vars})
            write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])  

#     for i in range(len(fbkeys)):
#         fb_vars = data.era5fb[fbkeys[i]][:]
#         fb_vars = fb_vars[absidx]
#         if fbkeys[i] == 'index' or fbkeys[i] == 'string6' or fbkeys[i] == 'string7' or fbkeys[i] == 'string10':
#             pass
#         elif fbkeys[i] == 'expver' or fbkeys[i] == 'source@hdr' or fbkeys[i] == 'source_id' or fbkeys[i] == 'statid@hdr':
#             alldict = {fbkeys[i]:np.asarray(fb_vars, dtype='S1')}
#             write_dict_h5(targetfile, alldict, 'era5fb', {fbkeys[i]: { 'compression': 'gzip' } }, [fbkeys[i]])
#         else:
#             alldict = pandas.DataFrame({fbkeys[i]:fb_vars})
#             write_dict_h5(targetfile, alldict, 'era5fb', {fbkeys[i]: { 'compression': 'gzip' } }, [fbkeys[i]]) 
    #
    # writing the recordindices and recordtimestamp.
    #       
    recordindices=vridx
    for i in range(len(recordindices)):
        testvar = pandas.DataFrame({str(allvars[i]):recordindices[i]})
        write_dict_h5(targetfile, testvar, 'recordindices', {str(allvars[i]): { 'compression': None } }, [str(allvars[i])]) 

    write_dict_h5(targetfile, {'recordtimestamp':rt}, 'recordindices', {'recordtimestamp': { 'compression': None } }, ['recordtimestamp'])

    print('elapsed:',time.time()-tt)



In [94]:
# with eua.CDMDataset('/raid60/scratch/leo/scratch/RI/Pangaea/nc/0-20100-0-00201.nc') as test:
#     print(test.observations_table)
# with eua.CDMDataset('/resorted_files_intercomp/0-20100-0-00201.nc') as test:
#     print(test)

In [89]:
do_resort('/raid60/scratch/leo/scratch/RI/Pangaea/nc/0-20100-0-00201.nc')

elapsed: 0.5056281089782715


In [95]:
if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=20)
    result_list = pool.map(do_resort, files[:])
    print(result_list)

# 4250:4300
# 4250:4255
# sonst alle bis 5500
# for i in range(len(files)):
#     do_resort(files[i])
#     if i > 2: break


elapsed: 1.5315845012664795
elapsed: 1.5997085571289062elapsed:
 elapsed:1.5588514804840088
 1.5607447624206543
elapsed: 1.568709135055542
elapsed: 1.5629076957702637
elapsed: 1.549851655960083
elapsed: 1.4894349575042725elapsed:
 1.5891351699829102
elapsed: 1.559189796447754
elapsed: 1.553767204284668
elapsed:elapsed:elapsed:  1.59126877784729 1.5946612358093262

1.6015119552612305
elapsed: 1.6234643459320068
elapsed: 1.6179451942443848
elapsed: 1.5299904346466064
elapsed: 1.6212189197540283
elapsed: 1.8305962085723877
elapsed: 1.792104721069336
elapsed: 1.4161343574523926
elapsed: 1.4175779819488525
elapsed: elapsed:1.401658058166504
 1.4413726329803467
elapsed: 1.318955659866333
elapsed: 1.4234223365783691
elapsed: 1.4309673309326172
elapsed: elapsed:1.4531896114349365 
elapsed:1.4446642398834229 1.4488966464996338
elapsed:
 1.443387746810913
elapsed: 1.517026424407959
elapsed: 1.5012574195861816
elapsed: 1.4731471538543701
elapsed: 1.430403470993042
elapsed: 1.4788234233856201
elap

In [92]:
with eua.CDMDataset('resorted_files_intercomp/0-20200-0-01502.nc') as data:
    print(data)

File: <HDF5 file "0-20200-0-01502.nc" (mode r)>
Filename: resorted_files_intercomp/0-20200-0-01502.nc
(G)roups/(V)ariables: 

 - G | crs__________________________________________ : : 4
 - V | dateindex____________________________________ : : (1, 3)
 - V | days_________________________________________ : : (1,)
 - V | drange_______________________________________ : : (3,)
 - G | header_table_________________________________ : : 46
 - G | id_scheme____________________________________ : : 4
 - G | iri_rstype_map_______________________________ : : 8
 - G | observations_table___________________________ : : 52
 - G | observed_variable____________________________ : : 15
 - V | record_______________________________________ : : (2,)
 - V | recordindex__________________________________ : : (2,)
 - G | recordindices________________________________ : : 4
 - V | recordtimestamp______________________________ : : (2,)
 - G | sensor_configuration_________________________ : : 12
 - G | source_configurat

In [17]:
files[4253:4254]

['/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-22165_CEUAS_merged_v0.nc']

In [20]:
files[4253]

'/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-22165_CEUAS_merged_v0.nc'

---

In [ ]:
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# # recordindices einzeln sortieren - für alle variablen eigene listen erstellen - dazu recordindices erstellen - alle daten hintereinander legen - len aller vorhergehender rec_in zu den nachkommenden addieren.
# #
# ri = data.recordindex[()]
# rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()
# recordindices = [[] for i in range(len(np.unique(allvars)))]
# recordtimestamps = [[] for i in range(len(np.unique(allvars)))]

# variables = [[] for i in range(len(np.unique(allvars)))]
# values = [[] for i in range(len(np.unique(allvars)))]
# zcoords = [[] for i in range(len(np.unique(allvars)))]

# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         break
        
#     a = data.observations_table.observed_variable[()][start:end]
#     b = data.observations_table.observation_value[()][start:end]
#     c = data.observations_table.z_coordinate[()][start:end]

#     sa, sb = zip(*sorted(zip(a, b)))
#     sa, sc = zip(*sorted(zip(a, c)))
#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 variables[j].append(sa[k])
#                 values[j].append(sb[k])
#                 zcoords[j].append(sc[k])
#         recordindices[j].append(len(variables[j]))
#         recordtimestamps[j].append(rt[i])
        
# lenadd = 0
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(variables[i])


In [ ]:
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# #
# ri = data.recordindex[()]
# rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()[:-1]
# recordindices = [[] for i in range(len(np.unique(allvars)))]
# recordtimestamps = [[] for i in range(len(np.unique(allvars)))]

# # output variables (from observations_table)
# ov = []
# for o in keys:
#     ov.append([[] for i in range(len(np.unique(allvars)))])
    
# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         break
        
#     a = data.observations_table.observed_variable[()][start:end]
#     helpvar = []
#     for o in range(len(keys)):
#         b = data.observations_table[keys[o]][()][start:end]
#         try: # all varibales with different shape won't work here
#             sa, sb = zip(*sorted(zip(a, b)))
#             helpvar.append(sb)
#         except:
#             pass

#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 for m in range(len(helpvar)):
#                     try: # all varibales with different shape won't work here
#                         ov[m][j].append(helpvar[m][k])
#                     except:
#                         pass
#         recordindices[j].append(len(ov[0][j]))
#         recordtimestamps[j].append(rt[i])
        
# lenadd = 0
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(variables[i])


In [ ]:
# #
# #
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# #
# ri = data.recordindex[()]
# print(len(ri))
# # rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()[:-1]
# # dropping all keys, where dimensions won't work - just help variabels for dimensions
# keys.pop(43)
# keys.pop(42)
# keys.pop(41)
# recordindices = [[] for i in range(len(allvars))]
# recordtimestamps = [[] for i in range(len(allvars))]

# # output variables (from observations_table)
# ov = []
# for o in keys:
#     ov.append([[] for i in range(len(np.unique(allvars)))])    
    
# print('resort:start')
# for i in range(len(ri)):
#     print(i)
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         start = ri[i]
#         end = len(data.observations_table.observed_variable[()])
        
#     a = data.observations_table.observed_variable[()][start:end]
#     helpvar = []
#     for o in range(len(keys)):
#         if keys[o] == 'observation_id' or keys[o] == 'report_id' or keys[o] == 'sensor_id' or keys[o] == 'source_id':
#             b = []
#             for n in data.observations_table[keys[o]][()][start:end]:
#                 c = ''
#                 for bb in n:
#                     c = c + bb.decode()
#                 b.append(c)
#         else:
#             b = data.observations_table[keys[o]][()][start:end]
#         sa, sb = zip(*sorted(zip(a, b)))
#         helpvar.append(sb)

#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 for m in range(len(helpvar)):
#                     ov[m][j].append(helpvar[m][k])
#         recordindices[j].append(len(ov[0][j]))
# #         recordtimestamps[j].append(rt[i])

# print('resort:done')
        
# #
# # setting record_indices to the right value -> stacking them
# #
# lenadd = 0.
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(ov[0][i])

# #
# # shaping record_indices -> setting every missing value to nan
# #
# old = -1
# for i in range(len(recordindices)):
#     for j in range(len(recordindices[i])):
#         if recordindices[i][j] != 0 and recordindices[i][j] == recordindices[i][j-1]:
#             old = recordindices[i][j]
#             recordindices[i][j] = np.nan
#         elif recordindices[i][j] == old:
#             recordindices[i][j] = np.nan

# #
# # recordtimestamps are only necessary once
# #
# recordtimestamps = recordtimestamps[0]

# # 
# # stacking all output variables
# #
# out = []
# for j in ov:
#     finalvar = []
#     for i in j:
#         finalvar = finalvar + i
#     out.append(finalvar)
    
# #
# # restoring byte arrays:
# #
# for o in range(len(keys)):
#     if keys[o] == 'observation_id' or keys[o] == 'report_id' or keys[o] == 'sensor_id' or keys[o] == 'source_id':
#         b = []
#         for n in range(len(out[o])):
#             c = []
#             for bb in out[o][n]:
#                 c.append(bb.encode())
#             out[o][n] = c


In [ ]:
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# #
# ri = data.recordindex[()]
# print('recordindex: ', len(ri))
# # rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()[:-1]
# # dropping all keys, where dimensions won't work - just help variabels for dimensions
# keys.pop(43)
# keys.pop(42)
# keys.pop(41)
# recordindices = [[] for i in range(len(allvars))]
# recordtimestamps = [[] for i in range(len(allvars))]

# # output variables (from observations_table)
# ov = []
# for o in keys:
#     ov.append([[] for i in range(len(np.unique(allvars)))])

# #
# # load data into memory and decode byte arrays
# #
# print('loading data')
# obsv = data.observations_table.observed_variable[:]
# ov_vars = []
# for o in range(len(keys)):
#     ov_vars.append(data.observations_table[keys[o]][:])
#     if keys[o] == 'observation_id' or keys[o] == 'report_id' or keys[o] == 'sensor_id' or keys[o] == 'source_id':
#         b = []
#         for n in ov_vars[o]:
#             c = ''
#             for bb in n:
#                 c = c + bb.decode()
#             b.append(c)
#         ov_vars[o] = b

# #
# # resorting the data
# #
# print('resort:start')
# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         start = ri[i]
#         end = len(data.observations_table.observed_variable[()])
        
#     a = obsv[start:end]
#     helpvar = []
#     for o in range(len(keys)):
#         b = ov_vars[o][start:end]
#         sa, sb = zip(*sorted(zip(a, b)))
#         helpvar.append(sb)

#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 for m in range(len(helpvar)):
#                     ov[m][j].append(helpvar[m][k])
#         recordindices[j].append(len(ov[0][j]))
# #         recordtimestamps[j].append(rt[i])

# print('resort:done')
        
# #
# # setting record_indices to the right value -> stacking them
# #
# lenadd = 0.
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(ov[0][i])

# #
# # shaping record_indices -> setting every missing value to nan
# #
# print('shaping record_indices')
# old = -1
# for i in range(len(recordindices)):
#     for j in range(len(recordindices[i])):
#         if recordindices[i][j] != 0 and recordindices[i][j] == recordindices[i][j-1]:
#             old = recordindices[i][j]
#             recordindices[i][j] = np.nan
#         elif recordindices[i][j] == old:
#             recordindices[i][j] = np.nan

# #
# # recordtimestamps are only necessary once
# #
# recordtimestamps = recordtimestamps[0]

# # 
# # stacking all output variables
# #
# print('stacking output variables')
# out = []
# for j in ov:
#     finalvar = []
#     for i in j:
#         finalvar = finalvar + i
#     out.append(finalvar)
    
# #
# # restoring byte arrays:
# # this takes very long -> find faster option
# #
# print('restoring byte arrays')
# bytelist = [25, 34, 38, 39]
# for o in bytelist:
#     originallen = len(data.observations_table[keys[o]][()][0])
#     for n in range(len(out[o])):
#         c = [elem.encode() for elem in out[o][n]]
#         # problem: if the string was [b'x', b'y', b''] in the first place, it get converted to 'xy' and then back to [b'x', b'y']
#         # add empty byte strings until the data is as long as before:
#         while(len(c) < originallen):
#             c.append(str('').encode())
#         out[o][n] = c

In [ ]:
# with eua.CDMDataset(files[0]) as data:
#     allvars = data.observations_table.observed_variable[()]
#     allvars.sort()
#     allvars = np.unique(allvars)
#     #
#     ri = data.recordindex[()]
#     print('recordindex: ', len(ri))
#     keys = data.observations_table.keys()[:-1]
#     # dropping all keys, where dimensions won't work - just help variabels for dimensions
#     keys.pop(43)
#     keys.pop(42)
#     keys.pop(41)
#     recordindices = [[] for i in range(len(allvars))]
#     recordtimestamps = [[] for i in range(len(allvars))]

#     # output variables (from observations_table)
#     ov = []
#     for o in keys:
#         ov.append([[] for i in range(len(np.unique(allvars)))])

#     #
#     # load data into memory and decode byte arrays
#     #
#     print('loading data')
#     obsv = data.observations_table.observed_variable[:]
#     ov_vars = []
#     for o in range(len(keys)):
#         ov_vars.append(data.observations_table[keys[o]][:])

# #
# # resorting the data
# #
# print('resort:start')
# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         start = ri[i]
#         end = len(obsv)
        
#     a = obsv[start:end]
#     sa, sortindex = zip(*sorted(zip(a, range(len(a)))))
#     sortindex = np.array(sortindex)
#     helpvar = []
#     for o in range(len(keys)):
#         b = ov_vars[o][start:end]
#         helpvar.append(b[sortindex])

#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 for m in range(len(helpvar)):
#                     ov[m][j].append(helpvar[m][k])
#         recordindices[j].append(len(ov[0][j]))

# print('resort:done')
        
# #
# # setting record_indices to the right value -> stacking them
# #
# lenadd = 0.
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(ov[0][i])

# #
# # shaping record_indices -> setting every missing value to nan
# #
# print('shaping record_indices')
# old = -1
# for i in range(len(recordindices)):
#     for j in range(len(recordindices[i])):
#         if i != 0 or j != 0:
#             if recordindices[i][j] == recordindices[i][j-1]:
#                 old = recordindices[i][j]
#                 recordindices[i][j] = np.nan
#             elif recordindices[i][j] == old:
#                 recordindices[i][j] = np.nan

# #
# # recordtimestamps are only necessary once
# #
# recordtimestamps = recordtimestamps[0]

# # 
# # stacking all output variables
# #
# print('stacking output variables')
# out = []
# for k in ov:
#     out.append([j for i in k for j in i])

In [ ]:
# def do_resort(fn):
    
#     data = eua.CDMDataset(fn)

#     allvars = data.observations_table.observed_variable[()]
#     allvars.sort()
#     allvars = numpy.unique(allvars)
#     #
#     ri = data.recordindex[()]
#     print('recordindex: ', len(ri))
#     rt = data.recordtimestamp[()]
#     keys = data.observations_table.keys()[:-1]
#     fbkeys = data.era5fb.keys()[:]
#     # dropping all keys, where dimensions won't work - just help variabels for dimensions
#     keys.pop(43)
#     keys.pop(42)
#     keys.pop(41)
#     recordindices = [[] for i in range(len(allvars))]
#     recordtimestamps = [[] for i in range(len(allvars))]

#     # output variables (from observations_table)
#     ov = []
#     for o in keys:
#         ov.append([[] for i in range(len(numpy.unique(allvars)))])

#     #
#     # load data into memory and decode byte arrays
#     #
#     print('loading data')
#     obsv = data.observations_table.observed_variable[:]
#     ov_vars = []
#     for o in range(len(keys)):
#         ov_vars.append(data.observations_table[keys[o]][:])

#     #
#     # resorting the data
#     #
#     print('resort:start')
#     @njit
#     def make_vrindex(vridx,ridx,idx):
#         l=0
#         for i in range(1,len(idx)): # to set the recordindices
#             if ridx[i]>ridx[i-1]:
#                 vridx[ridx[i-1]:ridx[i]]=l # next record after l
#                 l=i
#         vridx[ridx[i]:]=len(idx) # next record for the last element is the len of the data


#     tt=time.time()

#     ridxall=np.zeros(ov_vars[0].shape[0],dtype=np.int64) # reverse index - index of the record index
#     j=-1
#     for j in range(len(ri)-1):
#         ridxall[ri[j]:ri[j+1]]=j
#     j+=1
#     ridxall[ri[j]:]=j # for the last elemenet

#     ridx=[]
#     vridx=[]
#     absidx=[]
#     abscount=0
#     for j in range(len(allvars)):
#         idx=np.where(ov_vars[keys.index('observed_variable')]==allvars[j])[0] # index of all elements form certain variable j
#         print(j,len(idx),',',end='')
#         vridx.append(np.zeros(ri.shape[0],dtype=np.int64)) # all zeros in lenght of record index
#         ridx=ridxall[idx] # ridxall where variable is j
#         make_vrindex(vridx[-1],ridx,idx)
#         vridx[-1]+=abscount # abscount for stacking the recordindex

#         absidx.append(copy.copy(idx)) # why copy? - to make sure it's not just the ref. - maybe ok without the cp
#         abscount+=len(idx)

#     print('')
#     absidx=np.concatenate(absidx)
#     for o in range(len(keys)):
#         ov_vars[o]=ov_vars[o][absidx]
#         print(o,end='')
#     #
#     # recordtimestamps are only necessary once
#     #
#     recordtimestamps = recordtimestamps[0]


#     out = ov_vars
#     # targetfile has to be a copy of the original file
#     targetfile = '/raid60/scratch/uli/resorted_files_20201109/'+fn.split('/')[-1]# 0-20000-0-63894_CEUAS_merged_v0.nc'
#     if os.path.isfile(targetfile):
#         mode='r+'
#     else:
#         mode='w'
        
#     print()
#     print('writing '+targetfile)
#     #
#     # writing data into observations_table
#     #
#     with h5py.File(targetfile, mode) as file:
#         for i in range(len(keys)):
#             try:
#                 del file['observations_table'][keys[i]]
#             except:
#                 pass
#     for i in range(len(keys)):
#         if keys[i] == 'index':
#             pass
#         elif keys[i] == 'observation_id' or keys[i] == 'report_id' or keys[i] == 'sensor_id' or keys[i] == 'source_id':
# #             slen = len(out[i][0])
#             alldict = {keys[i]:np.asarray(out[i], dtype='S1')}
#             write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])
#         else:
#             alldict = pandas.DataFrame({keys[i]:out[i]})
#             write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])  
#         #
#         # writing the recordindices and recordtimestamp.
#         #       
#     recordindices=vridx
#     with h5py.File(targetfile, 'r+') as file:
#         for i in range(len(recordindices)):
#             try:
#                 del file['recordindices'][str(allvars[i])]
#             except:
#                 pass
#     for i in range(len(recordindices)):
#         testvar = pandas.DataFrame({str(allvars[i]):recordindices[i]})
#         write_dict_h5(targetfile, testvar, 'recordindices', {str(allvars[i]): { 'compression': None } }, [str(allvars[i])]) 

#     with h5py.File(targetfile, 'r+') as file:
#         try:
#             del file['recordindex']
#         except:
#             pass
#         try:
#             del file['recordtimestamp']
#         except:
#             pass
#         try:
#             del file['recordindices']['recordtimestamp']
#         except:
#             pass
#     write_dict_h5(targetfile, {'recordtimestamp':rt}, 'recordindices', {'recordtimestamp': { 'compression': None } }, ['recordtimestamp']) 
        
#     print('elapsed:',time.time()-tt)
#     print(end='')



# # In[15]:




In [3]:
# def do_resort(fn):
    
#     data = eua.CDMDataset(fn)

#     allvars = data.observations_table.observed_variable[()]
#     allvars.sort()
#     allvars = numpy.unique(allvars)
#     #
#     ri = data.recordindex[()]
#     print('recordindex: ', len(ri))
#     rt = data.recordtimestamp[()]
#     keys = data.observations_table.keys()[:-1]
#     fbkeys = data.era5fb.keys()[:-1]
#     # dropping all keys, where dimensions won't work - just help variabels for dimensions
#     pops = []
#     for i in range(len(keys)):
#         if 'string' in keys[i]:
#             pops.append(keys[i])
#     for i in pops: keys.remove(i)
#     pops = []
#     for i in range(len(fbkeys)):
#         if 'string' in fbkeys[i]:
#             pops.append(fbkeys[i])
#     for i in pops: fbkeys.remove(i)
        
#     recordindices = [[] for i in range(len(allvars))]
#     recordtimestamps = [[] for i in range(len(allvars))]

#     # output variables (from observations_table)
#     ov = []
#     for o in keys:
#         ov.append([[] for i in range(len(allvars))])
#     fb = []
#     for o in fbkeys:
#         fb.append([[] for i in range(len(allvars))])

#     #
#     # load data into memory and decode byte arrays
#     #
#     print('loading data')
#     obsv = data.observations_table.observed_variable[:]
#     ov_vars = []
#     for o in range(len(keys)):
#         ov_vars.append(data.observations_table[keys[o]][:])
#     fb_vars = []
#     for o in range(len(fbkeys)):
#         fb_vars.append(data.era5fb[fbkeys[o]][:])

#     #
#     # resorting the data
#     #
#     print('resort:start')
#     @njit
#     def make_vrindex(vridx,ridx,idx):
#         l=0
#         for i in range(1,len(idx)): # to set the recordindices
#             if ridx[i]>ridx[i-1]:
#                 vridx[ridx[i-1]:ridx[i]]=l # next record after l
#                 l=i
#         vridx[ridx[i]:]=len(idx) # next record for the last element is the len of the data


#     tt=time.time()

#     ridxall=np.zeros(ov_vars[0].shape[0],dtype=np.int64) # reverse index - index of the record index
#     j=-1
#     for j in range(len(ri)-1):
#         ridxall[ri[j]:ri[j+1]]=j
#     j+=1
#     ridxall[ri[j]:]=j # for the last elemenet

#     ridx=[]
#     vridx=[]
#     absidx=[]
#     abscount=0
#     for j in range(len(allvars)):
#         idx=np.where(ov_vars[keys.index('observed_variable')]==allvars[j])[0] # index of all elements form certain variable j
#         print(j,len(idx),',',end='')
#         vridx.append(np.zeros(ri.shape[0],dtype=np.int64)) # all zeros in lenght of record index
#         ridx=ridxall[idx] # ridxall where variable is j
#         make_vrindex(vridx[-1],ridx,idx)
#         vridx[-1]+=abscount # abscount for stacking the recordindex

#         absidx.append(copy.copy(idx)) # why copy? - to make sure it's not just the ref. - maybe ok without the cp
#         abscount+=len(idx)

#     print('')
#     absidx=np.concatenate(absidx)
#     for o in range(len(keys)):
#         ov_vars[o]=ov_vars[o][absidx]
#         print(o,end='')
        
#     for o in range(len(fbkeys)):
#         fb_vars[o]=fb_vars[o][absidx]
#         print(o,end='')

#     #
#     # recordtimestamps are only necessary once
#     #
#     recordtimestamps = recordtimestamps[0]


#     out = ov_vars
#     # targetfile has to be a copy of the original file
#     targetfile = '/raid60/scratch/uli/resorted_files_20201109/'+fn.split('/')[-1]# 0-20000-0-63894_CEUAS_merged_v0.nc'
#     if os.path.isfile(targetfile):
#         mode='r+'
#     else:
#         mode='w'
        
#     print()
#     print('writing '+targetfile)
#     #
#     # writing data into observations_table
#     #
#     with h5py.File(targetfile, mode) as file:
#         for i in range(len(keys)):
#             try:
#                 del file['observations_table'][keys[i]]
#             except:
#                 pass
#         for i in range(len(fbkeys)):
#             try:
#                 del file['era5fb'][fbkeys[i]]
#             except:
#                 pass
#     for i in range(len(keys)):
#         if keys[i] == 'index':
#             pass
#         elif keys[i] == 'observation_id' or keys[i] == 'report_id' or keys[i] == 'sensor_id' or keys[i] == 'source_id':
# #             slen = len(out[i][0])
#             alldict = {keys[i]:np.asarray(out[i], dtype='S1')}
#             write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])
#         else:
#             alldict = pandas.DataFrame({keys[i]:out[i]})
#             write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])  
            
#     for i in range(len(fbkeys)):
#         if fbkeys[i] == 'index' or fbkeys[i] == 'string6' or fbkeys[i] == 'string7' or fbkeys[i] == 'string10':
#             pass
#         elif fbkeys[i] == 'expver' or fbkeys[i] == 'source@hdr' or fbkeys[i] == 'source_id' or fbkeys[i] == 'statid@hdr':
# #             slen = len(out[i][0])
#             alldict = {fbkeys[i]:np.asarray(fb_vars[i], dtype='S1')}
#             write_dict_h5(targetfile, alldict, 'era5fb', {fbkeys[i]: { 'compression': 'gzip' } }, [fbkeys[i]])
#         else:
#             alldict = pandas.DataFrame({fbkeys[i]:fb_vars[i]})
#             write_dict_h5(targetfile, alldict, 'era5fb', {fbkeys[i]: { 'compression': 'gzip' } }, [fbkeys[i]]) 
#     #
#     # writing the recordindices and recordtimestamp.
#     #       
#     recordindices=vridx
#     with h5py.File(targetfile, 'r+') as file:
#         for i in range(len(recordindices)):
#             try:
#                 del file['recordindices'][str(allvars[i])]
#             except:
#                 pass
#     for i in range(len(recordindices)):
#         testvar = pandas.DataFrame({str(allvars[i]):recordindices[i]})
#         write_dict_h5(targetfile, testvar, 'recordindices', {str(allvars[i]): { 'compression': None } }, [str(allvars[i])]) 

#     with h5py.File(targetfile, 'r+') as file:
#         try:
#             del file['recordindex']
#         except:
#             pass
#         try:
#             del file['recordtimestamp']
#         except:
#             pass
#         try:
#             del file['recordindices']['recordtimestamp']
#         except:
#             pass
#     write_dict_h5(targetfile, {'recordtimestamp':rt}, 'recordindices', {'recordtimestamp': { 'compression': None } }, ['recordtimestamp']) 
        
#     print('elapsed:',time.time()-tt)
#     print(end='')



# # In[15]:




In [ ]:
# def do_resort(fn):
#     targetfile = '/raid60/scratch/uli/resorted_files/'+fn.split('/')[-1]  
    
#     with h5py.File(fn, 'r') as file:
#         with h5py.File(targetfile, 'w') as newfile:
#             newfile.copy(file['header_table'],newfile)
#             newfile.copy(file['sensor_configuration'],newfile)
#             newfile.copy(file['crs'],newfile)
#             newfile.copy(file['observed_variable'],newfile)
#             newfile.copy(file['source_configuration'],newfile)
#             newfile.copy(file['station_configuration'],newfile)
#             newfile.copy(file['station_type'],newfile)
#             newfile.copy(file['station_configuration_codes'],newfile)
#             newfile.copy(file['units'],newfile)
#             newfile.copy(file['z_coordinate_type'],newfile)
            
#             newfile.create_dataset('dateindex', data=file['dateindex'][:]) 

    
#     data =  eua.CDMDataset(fn)
#     allvars = data.observations_table.observed_variable[()]
#     allvars.sort()
#     allvars = numpy.unique(allvars)
#     #
#     ri = data.recordindex[()]
# #     print('recordindex: ', len(ri))
#     rt = data.recordtimestamp[()]
#     keys = data.observations_table.keys()[:-1]
#     fbkeys = data.era5fb.keys()[:-1]
#     # dropping all keys, where dimensions won't work - just help variabels for dimensions
#     pops = []
#     for i in range(len(keys)):
#         if 'string' in keys[i]:
#             pops.append(keys[i])
#     for i in pops: keys.remove(i)
#     pops = []
#     for i in range(len(fbkeys)):
#         if 'string' in fbkeys[i]:
#             pops.append(fbkeys[i])
#     for i in pops: fbkeys.remove(i)

#     recordindices = [[] for i in range(len(allvars))]
#     recordtimestamps = [[] for i in range(len(allvars))]

#     # output variables (from observations_table)
#     ov = []
#     for o in keys:
#         ov.append([[] for i in range(len(allvars))])
#     fb = []
#     for o in fbkeys:
#         fb.append([[] for i in range(len(allvars))])
#     #
#     # loading the observed_variables
#     #
#     obsv = data.observations_table.observed_variable[:]
#     #
#     # resorting the data
#     #
# #     print('resort:start')
#     @njit
#     def make_vrindex(vridx,ridx,idx):
#         l=0
#         for i in range(1,len(idx)): # to set the recordindices
#             if ridx[i]>ridx[i-1]:
#                 vridx[ridx[i-1]:ridx[i]]=l # next record after l
#                 l=i
#         vridx[ridx[i]:]=len(idx) # next record for the last element is the len of the data


#     tt=time.time()

#     ridxall=np.zeros(obsv.shape[0],dtype=np.int64) # reverse index - index of the record index
#     j=-1
#     for j in range(len(ri)-1):
#         ridxall[ri[j]:ri[j+1]]=j
#     j+=1
#     ridxall[ri[j]:]=j # for the last elemenet
#     ridx=[]
#     vridx=[]
#     absidx=[]
#     abscount=0
#     for j in range(len(allvars)):
#         idx=np.where(obsv==allvars[j])[0] # index of all elements form certain variable j
# #         print(j,len(idx),',',end='')
#         vridx.append(np.zeros(ri.shape[0],dtype=np.int64)) # all zeros in lenght of record index
#         ridx=ridxall[idx] # ridxall where variable is j
#         make_vrindex(vridx[-1],ridx,idx)
#         vridx[-1]+=abscount # abscount for stacking the recordindex

#         absidx.append(copy.copy(idx)) # why copy? - to make sure it's not just the ref. - maybe ok without the cp
#         abscount+=len(idx)

# #     print('')
#     #
#     # finishing the sorting 
#     #
#     absidx=np.concatenate(absidx)
#     #
#     # recordtimestamps are only necessary once
#     #
#     recordtimestamps = recordtimestamps[0]
#     #
#     # targetfile has to be a copy of the original file
#     #
#     targetfile = '/raid60/scratch/uli/resorted_files/'+fn.split('/')[-1]# 0-20000-0-63894_CEUAS_merged_v0.nc'
#     if os.path.isfile(targetfile):
#         mode='r+'
#     else:
#         mode='w'
# #     print()
# #     print('writing '+targetfile)

#     #
#     # writing data into observations_table
#     #
# #     with h5py.File(targetfile, mode) as file:
# #         for i in range(len(keys)):
# #             try:
# #                 del file['observations_table'][keys[i]]
# #             except:
# #                 pass
# #         for i in range(len(fbkeys)):
# #             try:
# #                 del file['era5fb'][fbkeys[i]]
# #             except:
# #                 pass
#     for i in range(len(keys)):
#         ov_vars = data.observations_table[keys[i]][:]
#         ov_vars = ov_vars[absidx]
#         if keys[i] == 'index':
#             pass
#         elif keys[i] == 'observation_id' or keys[i] == 'report_id' or keys[i] == 'sensor_id' or keys[i] == 'source_id':
#             alldict = {keys[i]:np.asarray(ov_vars, dtype='S1')}
#             write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])
#         else:
#             alldict = pandas.DataFrame({keys[i]:ov_vars})
#             write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])  

#     for i in range(len(fbkeys)):
#         fb_vars = data.era5fb[fbkeys[i]][:]
#         fb_vars = fb_vars[absidx]
#         if fbkeys[i] == 'index' or fbkeys[i] == 'string6' or fbkeys[i] == 'string7' or fbkeys[i] == 'string10':
#             pass
#         elif fbkeys[i] == 'expver' or fbkeys[i] == 'source@hdr' or fbkeys[i] == 'source_id' or fbkeys[i] == 'statid@hdr':
#             alldict = {fbkeys[i]:np.asarray(fb_vars, dtype='S1')}
#             write_dict_h5(targetfile, alldict, 'era5fb', {fbkeys[i]: { 'compression': 'gzip' } }, [fbkeys[i]])
#         else:
#             alldict = pandas.DataFrame({fbkeys[i]:fb_vars})
#             write_dict_h5(targetfile, alldict, 'era5fb', {fbkeys[i]: { 'compression': 'gzip' } }, [fbkeys[i]]) 
#     #
#     # writing the recordindices and recordtimestamp.
#     #       
#     recordindices=vridx
# #     with h5py.File(targetfile, 'w') as file:
# #         for i in range(len(recordindices)):
# #             try:
# #                 del file['recordindices'][str(allvars[i])]
# #             except:
# #                 pass
#     for i in range(len(recordindices)):
#         testvar = pandas.DataFrame({str(allvars[i]):recordindices[i]})
#         write_dict_h5(targetfile, testvar, 'recordindices', {str(allvars[i]): { 'compression': None } }, [str(allvars[i])]) 

# #     with h5py.File(targetfile, 'w') as file:
# #         try:
# #             del file['recordindex']
# #         except:
# #             pass
# #         try:
# #             del file['recordtimestamp']
# #         except:
# #             pass
# #         try:
# #             del file['recordindices']['recordtimestamp']
# #         except:
# #             pass
#     write_dict_h5(targetfile, {'recordtimestamp':rt}, 'recordindices', {'recordtimestamp': { 'compression': None } }, ['recordtimestamp'])
# #     with h5py.File(targetfile, 'w') as newfile:
# #         for i in data.header_table.keys():
# #             alldict = pandas.DataFrame({i:data.header_table[i]})
# #             write_dict_h5(targetfile, alldict, 'header_table', {i: { 'compression': 'gzip' } }, [i])

#     print('elapsed:',time.time()-tt)
# #     print(end='')



# # In[15]:


